# Combinação de classificadores

In [1]:
import pickle

In [2]:
with open('../../assets/credit.pkl', 'rb') as f:
  X_credit_train, y_credit_train, X_credit_test, y_credit_test = pickle.load(f)

In [3]:
import numpy as np

In [4]:
X_credit = np.concatenate((X_credit_train, X_credit_test), axis = 0)
y_credit = np.concatenate((y_credit_train, y_credit_test), axis = 0)

In [5]:
neural_network = pickle.load(open('rede_neural_finalizado.sav', 'rb'))
tree = pickle.load(open('arvore_finalizado.sav', 'rb'))
svm = pickle.load(open('svm_finalizado.sav', 'rb'))

In [6]:
new_record = X_credit[1999]
new_record

array([-1.03572293, -0.93978209,  0.04244312])

In [7]:
new_record.shape

(3,)

In [8]:
new_record = new_record.reshape(1, -1)
new_record.shape

(1, 3)

In [9]:
new_record

array([[-1.03572293, -0.93978209,  0.04244312]])

In [10]:
result_neural_network = neural_network.predict(new_record)

In [11]:
result_tree = tree.predict(new_record)

In [12]:
result_svm = svm.predict(new_record)

In [13]:
results = np.concatenate((result_neural_network, result_tree, result_svm))

In [14]:
pay = (results == 0).sum()
not_pay = (results == 1).sum()

pay, not_pay

(0, 3)

In [15]:
if pay > not_pay:
  print("Cliente pagará o empréstimo")
elif pay == not_pay:
  print("Empate")
else:
  print("Cliente não pagará o empréstimo")

Cliente não pagará o empréstimo


# Rejeição de classificação

In [16]:
prob_neural_network = neural_network.predict_proba(new_record)
prob_neural_network

array([[1.33098109e-04, 9.99866902e-01]])

In [19]:
trust_neural_network = prob_neural_network.max()
trust_neural_network

0.9998669018912271

In [22]:
prob_tree = tree.predict_proba(new_record)
trust_tree = prob_tree.max()
trust_tree

1.0

In [23]:
prob_svm = svm.predict_proba(new_record)
trust_svm = prob_svm.max()
trust_svm

0.9918957645462223

# Refatoração

In [26]:
algorithms = {
    "neural_network": [neural_network],
    "tree": [tree],
    "svm": [svm]
}

for algorithm, data in algorithms.items():
  model = data[0]

  result = model.predict(new_record)
  prob = model.predict_proba(new_record)
  trust = prob.max()

  data += [result, prob, trust]
  

In [29]:
trust_limiar = 0.999999
count = 0
not_pay, pay = 0, 0

for data in algorithms.values():
    if data[3] < trust_limiar:
        continue
    
    if data[1] == 1:
        not_pay += 1
        count += 1

    if data[1] == 0:
        pay += 1
        count += 1

if pay > not_pay:
    print(f"Cliente pagará o empréstimo, baseado em {count} algoritmos.")
elif pay == not_pay:
    print(f"Empate, baseado em {count} algoritmos.")
else:
    print(f"Cliente não pagará o empréstimo, baseado em {count} algoritmos.")

Cliente não pagará o empréstimo, baseado em 1 algoritmos.
